In [2]:
import numpy as np 
import pandas as pd 
import os
import csv
import warnings
warnings.simplefilter(action='ignore')
import gc
from pathlib import Path 

In [3]:
df_combined = pd.read_csv('sampled_dataset_full_column.csv')

columns = ["userid","username","location","tweetid","tweetcreatedts","retweetcount", "hashtags", "text", "usercreatedts", "following", "followers","totaltweets"]

# select equal timeframe bafore and after the ban of twitter in Russia
df1 = df_combined[columns][df_combined['tweetcreatedts'] > "2022-02-20 00:00:00" ] 
df1 = df1[df1['tweetcreatedts'] < "2022-03-14 00:00:00" ] 
df1.shape

(53158, 12)

In [4]:
df1.sample(3)

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets
430643,265307837,makhiscal,Federación Matriarcal Mhanka,1.499537e+18,2022-03-04 00:07:00,20.0,"[{'text': 'FLASH', 'indices': [12, 18]}, {'tex...",#FLASH—Fierce fighting on the approaches to th...,2011-03-13 10:12:41.000000,348.0,164.0,24914.0
424927,1263425924613107712,tigerwa70250694,NaN,1.498826e+18,2022-03-02 01:03:54,25.0,"[{'text': 'China', 'indices': [118, 124]}]",Talking with @MariaBartiromo on @MorningsMaria...,2020-05-21 11:06:43.000000,45.0,269.0,101688.0
417955,3807464175,claracampostor1,NaN,1.498289e+18,2022-02-28 13:28:59,38.0,"[{'text': 'Ukraine', 'indices': [35, 43]}, {'t...",Russia's invasion of #Ukraine is criminal.\nHe...,2015-09-28 21:00:05.000000,434.0,1367.0,286357.0


### Hashtags

### (1) top used hashtags 

In [5]:
tags = df1['hashtags'].tolist()

TAGS=[]
for item in tags:
    TAGS+=item[1:-1].split('{')
TAGS2=[]
for item2 in TAGS:
    if item2!='':
        TAGS2+=[item2.split(': ')[1].split(',')[0][1:-1]]
tagdf=pd.DataFrame(pd.Series(TAGS2).value_counts())
tagdf['tag']=tagdf.index.tolist()
tagdf.columns=['counts','tag']
tagdf.index=range(len(tagdf))
tagdf[1:20]

#tag_c = tagdf[tagdf['counts']>30]
#tag_c.to_csv("hashtag_count_over30.csv",sep = ",", header = True, encoding = "UTF-8",index=False)

In [6]:
tag_c.sample(6)

,counts,tag
88,111,UN
242,31,Ucraina
55,214,IStandWithPutin
207,40,cryptocurrency
125,71,Kremlin
98,99,NoFlyZoneOverUkraine


### (2) clean hashtags for each tweet 

In [7]:
# write a function to extract "hashtags" 
def SplitTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
    if len(TAGS)>0:
        return TAGS
    else: return "NA"

def LowerTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
    lower_TAGS = (map(lambda x: x.lower(), TAGS))
    TAGS = list(lower_TAGS)
    if len(TAGS)>0:
        return TAGS
    else: return "NA"

            
df1['tags_splitted'] = df1['hashtags'].apply(SplitTags)
df1['tags_lower'] = df1['hashtags'].apply(LowerTags)
df1.sample(3)

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower
441369,2423403146,Santiag87507989,NaN,1.500937e+18,2022-03-07 20:51:21,5090.0,[],"All Ukrainian, you are great heroes! Today you...",2014-04-02 07:38:45.000000,935.0,58.0,21931.0,NA,NA
438046,965384533376995329,MariaSchenetzke,"Little Rock, AR",1.500262e+18,2022-03-06 00:06:33,58.0,"[{'text': 'StandWithUkraine', 'indices': [111,...",The hacker collective Anonymous International ...,2018-02-19 00:36:23.000000,1653.0,1352.0,167250.0,[StandWithUkraine],[standwithukraine]
421336,1351169758054514696,Sarthak69516490,"Dehradun, India",1.498728e+18,2022-03-01 18:31:07,0.0,"[{'text': 'RussiaUkraine', 'indices': [253, 26...","So Ukraine will thrown under the bus, Ukraine ...",2021-01-18 14:09:19.000000,1.0,11.0,63.0,[RussiaUkraine],[russiaukraine]


### (3) create node attribute of political orientation

In [23]:
# import hashtag file 
df_tags = pd.read_csv("hashtag_over30_tagged.csv")

pro_R_tags = df_tags[df_tags["Pro_R"]=="TRUE"]['tag'].to_list()
pro_U_tags = df_tags[df_tags["Pro_U"]]['tag'].to_list()
Neu_tags = df_tags[df_tags["Neutral"]]['tag'].to_list()

pd.crosstab(index=df_tags['Pro_R'], columns='count') # 5
pd.crosstab(index=df_tags['Pro_U'], columns='count') #61
pd.crosstab(index=df_tags['Neutral'], columns='count') #178
# remember to use this in updated code
df_tags.sample(5)

,counts,tag,Pro_U,Pro_R,Neutral
211,106,IStandWithUkraine,True,FALSE,False
172,31,Sikhsoldiers,False,FALSE,True
79,71,SWIFT,False,FALSE,True
107,54,Luhansk,False,FALSE,True
108,54,Khalistan,False,FALSE,True


In [24]:
pro_R_tags_ = ["IStandWithPutin","istandwithrussia","ISupportRussia","ZelenskyWarCriminal","Zenlenskyisawarcriminal",\
              "NaziUkraine","UkraineNazis","nazisinukraine","WarCrimesOfUkraine","PutinIsRight","StopHatingRussians",\
              "NATOWarCriminal", "PatrioticRussia"]
pro_U_tags_ = ["Istandwithukraine","Putinisawarcriminal","PutinWarCriminal","StandWithUkraine", "HandsOffUkraine",\
              "kievfree","UkraineUnderAttack","StopRussia","StopPutin","RussiaInvadedUkriane","StopPutinsWar",\
              "StopRussianAggression","ukraineinvasion","russianinvasion","RussiaInvadedUkraine","PrayingForUkraine",\
              "RussiaOutOfUkraine","StandWithUkriane","RussiaInvadesUkraine","PutinHitler"]

pro_R_tags += pro_R_tags_
pro_U_tags += pro_U_tags_

# change all tags to lowercase before matching to avoid spelling difference
lower_R = (map(lambda x: x.lower(), pro_R_tags))
pro_R_tags = list(lower_R)

lower_U = (map(lambda x: x.lower(), pro_U_tags))
pro_U_tags = list(lower_U)

lower_N = (map(lambda x: x.lower(), Neu_tags))
Neu_tags = list(lower_N)

# lambda functions marking political orientation if related hashtags were found 
pro_R_check = lambda taglist: bool(set(taglist) & set(pro_R_tags)) # return true if overlap of two sets not empty
pro_U_check = lambda taglist: bool(set(taglist) & set(pro_U_tags)) 
N_check = lambda taglist: bool(set(taglist) & set(Neu_tags)) 


df1['pro_Russian'] = df1['tags_lower'].apply(pro_R_check)
df1['pro_Ukraine'] = df1['tags_lower'].apply(pro_U_check)
df1['Neutral'] = df1['tags_lower'].apply(N_check)


In [25]:
df1.head()

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower,pro_Russian,pro_Ukraine,Neutral
408402,879477038599487488,felionaaaa,NaN,1.497216e+18,2022-02-25 14:24:32,261.0,"[{'text': 'Ukraine', 'indices': [67, 75]}, {'t...",President of ukraine is leading from the front...,2017-06-26 23:10:40.000000,117.0,15.0,4482.0,"[Ukraine, UkraineRussia, Putin, UkraineInvasio...","[ukraine, ukrainerussia, putin, ukraineinvasio...",False,True,True
408403,910979270054338561,Patrici25530591,NaN,1.497175e+18,2022-02-25 11:40:25,41635.0,[],.@ZelenskyyUa's tv address to the Russian (!) ...,2017-09-21 21:29:18.000000,61.0,32.0,3959.0,NA,NA,False,False,False
408404,983264761,eantonicelli,"Stavanger, Norway",1.497525e+18,2022-02-26 10:50:58,150.0,[],The Ukrainian civilians who were handed out 18...,2012-12-01 20:15:21.000000,2880.0,2590.0,19074.0,NA,NA,False,False,False
408405,1144411063984828418,YoDammit,VA,1.497778e+18,2022-02-27 03:38:45,3396.0,"[{'text': 'Russia', 'indices': [59, 66]}]",⚡ Intel and AMD stop supplying microchips to #...,2019-06-28 01:03:54.000000,79.0,33.0,5521.0,[Russia],[russia],False,False,True
408406,2369760817,CharleyBurt1,USA,1.497116e+18,2022-02-25 07:46:59,563.0,[],"I’m Russian, and I want you to know that the R...",2014-03-03 03:13:04.000000,3351.0,4112.0,46789.0,NA,NA,False,False,False


In [26]:
no_tag = df1[df1['tags_splitted']=="NA"]
# 10594 tweets with no hashtags

# 255 labeled as pro-Russia
proRussia = df1[df1['pro_Russian']==True]
proRussiaNoUkraine = df1[(df1['pro_Russian']==True) & (df1['pro_Ukraine']==False)]

# 6412 labeled as pro-Ukraine
proUkraine = df1[df1['pro_Ukraine']==True]
proUkraineNoRussia = df1[(df1['pro_Russian']==False) & (df1['pro_Ukraine']==True)]

# still a lot rabeled by neither of two groups
non_R = df1[df1['pro_Russian']==False]
# non = non_R[non_R['pro_Ukraine']==False]

non = df1[(df1['pro_Russian']==False) & (df1['pro_Ukraine']==False)]
both = df1[(df1['pro_Russian']==True) & (df1['pro_Ukraine']==True)]

display(proRussia.shape)
display(proUkraine.shape)
display(non.shape)
display(both.shape)
display(proRussiaNoUkraine.shape)
display(proUkraineNoRussia.shape)


(974, 17)

(9595, 17)

(42885, 17)

(296, 17)

(678, 17)

(9299, 17)

In [27]:
both["text"].to_list()

['Breaking | Ukraine Under Attack | Hint News\nhttps://t.co/qdzwX1xNhc\n#HintNews #StandWithUkraine  #UkraineWar #Russia #RussiaNews #Ukraine #Europe  #UkraineTensions #UK #BorrisJohnson #UkraineNews #UkraineCrisis #RussiaUkraine #UkraineConflict #InternationalNews #globalnews #India',
 'Wreckage Falls Over Kyiv #Kyiv #Ukraine #UkraineRussia #UkraineInvasion #UcraniaRussia #RussiaUkraineConflict #Russia #RussiaInvadedUkraine #USA #NATO #Kiev https://t.co/bBwKvkrJBK',
 'A picture is worth a thousand words 🔥\n\n#WWIII #RussiaUkraineConflict #NATO #Putin #worldwar3 #StopWar #Russia #Ukraine https://t.co/hK1IOidJ4m',
 'Andhra Pradesh opens helplines for students stranded in Ukraine\n#AndhraPradesh #student #studentlife #Helpline\n#UkraineCrisis #UkraineConflict #IndiaNews\nhttps://t.co/guucoKxvl2',
 'This is not ok y’all…. #WARINUKRAINE #UkraineRussia #RussiaUkraineConflict #แตงโม #BREAKING #Ukraine https://t.co/6nwRcXz32V',
 'Russian students studying in the US and Western Europe have bee

In [12]:
df_R = non[non['text'].str.contains("zelensky" and "greenscreen")]
# tried keywords “Zenlensky” AND “Greenscreen” but no results in the non-labeled
df_R.shape

(0, 16)

### Nodelist with political stance

In [28]:
node_columns = ["userid","username","usercreatedts","following","followers","totaltweets","pro_Russian","pro_Ukraine","Neutral"]
nodesBefore = df1[node_columns][(df1['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
nodesAfter = df1[node_columns][(df1['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]

difAll = round(len(nodesAfter) * 100 / len(nodesBefore), 2)
print("Tweets before Russia's block:", len(nodesBefore), "and after:", len(nodesAfter), "\nDifference:", difAll)

proRussianBefore = proRussia[node_columns][(proRussia['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
proRussianAfter = proRussia[node_columns][(proRussia['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]
difRus = round(len(proRussianAfter) * 100 / len(proRussianBefore),2)
print("Pro-Russian tweets before Russia's block:", len(proRussianBefore), "and after:", len(proRussianAfter), "\nDifference:", difRus)

ratioBefore = round(len(proRussianBefore)/len(nodesBefore), 4)
ratioAfter = round(len(proRussianAfter)/len(nodesAfter), 4)

print("Ratio pro-Russian:Total tweets before block:", ratioBefore, "and after:", ratioAfter)


Tweets before Russia's block: 23156 and after: 30002 
Difference: 129.56
Pro-Russian tweets before Russia's block: 680 and after: 294 
Difference: 43.24
Ratio pro-Russian:Total tweets before block: 0.0294 and after: 0.0098


In [29]:
# code for nodetype variable

# Determine a user type by it's hashtags: pro-russian / pro-ukrainian / unknown
nodesBefore['NodeType'] = np.where(
     nodesBefore['pro_Russian'] & nodesBefore['pro_Ukraine'], 'Both',
     np.where(
         nodesBefore['pro_Russian'],  'pro_Russian',
     np.where(
        nodesBefore['pro_Ukraine'], 'pro_Ukraine',
    np.where(nodesBefore['Neutral'], 'Neutral', 'Unknown'
                ))))

nodesAfter['NodeType'] = np.where(
    nodesAfter['pro_Russian'] & nodesAfter['pro_Ukraine'], 'Both',
    np.where(
    nodesAfter['pro_Russian'], 'pro_Russian', 
    np.where(
        nodesAfter['pro_Ukraine'], 'pro_Ukraine', 
    np.where(nodesAfter['Neutral'], 'Neutral', 'Unknown'
                ))))

In [30]:
PUB = len(nodesBefore[nodesBefore["NodeType"] == "pro_Ukraine"])
PUA = len(nodesAfter[nodesAfter["NodeType"] == "pro_Ukraine"])
PRB = len(nodesBefore[nodesBefore["NodeType"] == "pro_Russian"])
PRA = len(nodesAfter[nodesAfter["NodeType"] == "pro_Russian"])
print(PRB+PRA)
# Need to eliminate

678


In [31]:
#pd.crosstab(index=nodelist['pro_Russian'], columns='count')
x = pd.crosstab(index=proRussianBefore['username'], columns='count')
print(x[x["count"] > 1])
x = pd.crosstab(index=proRussianAfter['username'], columns='count')
print(x[x["count"] > 1])

col_0           count
username             
BitcoinSurge        2
HungoaClub07        2
ORFMumbai           2
ProSportsExtra      3
TheCombat_Zone      2
TopFeaturedcom      3
_frnow              2
bulenta78           2
forevernews_in      2
tindian88           2
col_0            count
username              
Free_Libyan2022      3
KamaroMarie          2
KellyY67122045       2
Level_Roshanbir      2
MovieXen             3
Real_Jayant          2
aficionali           2
gulf_news            2
tngsmama             2


In [32]:
#merge with node in the retweet projected graph
projected_w_user_edges_2 = pd.read_csv("projected_w_user_edgelist_1.csv")
graph1_nodes = set(projected_w_user_edges_2["Source"])
nodes1 = set(nodesBefore["userid"].apply(lambda x: int(x))) # string -> int
overlap1 = list(graph1_nodes & nodes1)

projected_w_user_edges_2 = pd.read_csv("projected_w_user_edgelist_2.csv")
graph2_nodes = set(projected_w_user_edges_2["Target"])
nodes2 = set(nodesAfter["userid"].apply(lambda x: int(x)))
overlap2 = list(graph2_nodes & nodes2)
#len(overlap2)

In [33]:
# mark the nodes appeared in the projected network

# lambda functions marking political orientation if related hashtags were found 
node1_check = lambda node: int(node) in overlap1 # return true if userid of the nodes in overlap list
node2_check = lambda node: int(node) in overlap2

nodesBefore['nodeInG'] = nodesBefore['userid'].apply(node1_check)
nodesAfter['nodeInG'] = nodesAfter['userid'].apply(node2_check)

nodesBefore["ID"] = nodesBefore["userid"].apply(lambda x: int(x))
nodesAfter["ID"] = nodesAfter["userid"].apply(lambda x: int(x))

nodelist1 = nodesBefore[nodesBefore['nodeInG']]
nodelist2 = nodesAfter[nodesAfter['nodeInG']]


In [34]:
nodelist1.to_csv("nodelist1_poli.csv",sep = ",", header = True, encoding = "UTF-8",index=False)
nodelist2.to_csv("nodelist2_poli.csv",sep = ",", header = True, encoding = "UTF-8",index=False)

### Edgelist for user->hashtag mapping

In [ ]:
hashtags = df1['tags_splitted'][0]


['Ukraine', 'UkraineRussia', 'Putin', 'UkraineInvasion', 'Ukrainian', 'แตงโม']

In [ ]:
def UHmapping(hashtags):
    TAGS=[]
    if hashtags != 'NA':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]

edgelist['hashtag'] = df1['hashtags'].apply(UHmapping)
                          

                          

In [ ]:
edgelist = {'username':[],'tag':[]}
#df1 = df1.reset_index()  # make sure indexes pair with number of rows
for index, row in df1.iterrows():
    user = row['username']
    hashtags = row['tags_splitted']
    # change pandas series to list
    if hashtags != "NA":
        hashtags = list(pd.Series(hashtags).values)
        for tag in hashtags:
            edgelist.setdefault('username',[]).append(user)
            edgelist.setdefault('tag',[]).append(tag) 

edgelist = pd.DataFrame.from_dict(edgelist)

In [ ]:
edgelist

,username,tag
0,felionaaaa,Ukraine
1,felionaaaa,UkraineRussia
2,felionaaaa,Putin
3,felionaaaa,UkraineInvasion
4,felionaaaa,Ukrainian
...,...,...
108769,bigwincam,UKRAINE
108770,liontwee,Putin
108771,liontwee,StopPutinsWar
108772,liontwee,PutinIsaWarCriminal


In [ ]:
edgelist.to_csv("edgelist_user_tag.csv",sep = ",", header = True, encoding = "UTF-8",index=False)